In [22]:
import json
import os
import torch
import numpy as np
from unsloth import FastLanguageModel
from loguru import logger
from datasets import Dataset
# 导入 SFTTrainer, TrainingArguments, และ EvalPrediction
from transformers import TrainingArguments, EvalPrediction
from trl import SFTTrainer

# 用户的自定义日志模块，保持不变
from logServer import logServer
logServer().set_config(file_log_level="DEBUG", console_log_level="DEBUG")

logger.info("开始模型微调流程")

# 1. 加载原始数据集
dataset_list = []
try:
    with open('/hy-tmp/慢性病判别数据集.txt', 'r', encoding='utf-8') as f:
        for line in f:
            dataset_list.append(json.loads(line))
    logger.info(f"已加载数据集，包含 {len(dataset_list)} 个样本")
except FileNotFoundError:
    logger.error("数据集文件未找到，请检查路径 /hy-tmp/慢性病判别数据集.txt 是否正确。")
    exit()

# 2. 加载模型和分词器
MAX_SEQ_LENGTH = 512
logger.info(f"正在从 /hy-tmp/Qwen3-14b 加载模型，最大序列长度: {MAX_SEQ_LENGTH}")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="/hy-tmp/Qwen3-14b",
    load_in_4bit=True,
    local_files_only=True,
    max_seq_length=MAX_SEQ_LENGTH,
)
logger.info("模型和分词器加载成功")

# 3. 配置 LoRA
logger.info("正在配置 LoRA 适配器")
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    use_gradient_checkpointing=True,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
)
logger.info("LoRA 适配器配置完成")

# 4. 数据预处理与划分
EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(example):
    """为单个样本创建标准格式的提示文本。"""
    example["text"] = (
        f"输入: {example['text']}\n"
        f"输出: label: {example['label']}, category: {example['category']}, certainty: {example['certainty']}"
        f"{EOS_TOKEN}"
    )
    return example

full_dataset = Dataset.from_list(dataset_list)
dataset_split = full_dataset.train_test_split(test_size=0.1, seed=42)

train_dataset = dataset_split["train"].map(formatting_prompts_func)
eval_dataset = dataset_split["test"].map(formatting_prompts_func)
logger.info(f"数据集已划分为 {len(train_dataset)} 条训练样本和 {len(eval_dataset)} 条验证样本。")

# --- 新增部分：定义自定义评估指标计算函数 ---
def compute_metrics(eval_pred: EvalPrediction):
    """
    自定义评估指标计算函数。
    :param eval_pred: Trainer 在评估时传入的对象，包含模型的预测和真实标签。
    :return: 一个包含指标名称和值的字典。
    """
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    # 忽略标签为-100的部分（通常是prompt和padding）
    mask = labels != -100
    accuracy = np.mean(predictions[mask] == labels[mask])
    return {"accuracy": accuracy}
# --- ---

# 5. 配置训练参数 (最终版)
training_args = TrainingArguments(
    output_dir="/hy-tmp/Train_logs",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    optim="adamw_8bit",
    seed=3407,
    weight_decay=0.01,
    lr_scheduler_type="linear",
    warmup_steps=10,
    logging_steps=10,
    bf16=True,
    report_to="none",
    
    # 评估与保存策略
    do_eval=True,
    eval_steps=100,
    save_steps=100,
    save_total_limit=2,
    
    # 禁用了 load_best_model_at_end 以绕过版本兼容性问题
)

# 6. 初始化 SFTTrainer
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset, 
    tokenizer=tokenizer,
    args=training_args,
    dataset_text_field="text",
    max_seq_length=MAX_SEQ_LENGTH,
    packing=False,
    compute_metrics=compute_metrics, # **修改点**：传入我们自定义的评估函数
)

# 7. 开始训练
logger.info("准备就绪，使用 SFTTrainer 开始带有验证的训练流程")
trainer.train()

# 8. 训练完成并手动加载和保存最优模型
logger.info("训练完成")

logs = trainer.state.log_history
# 找出所有包含 'eval_loss' 和 'eval_accuracy' 的评估日志
eval_logs = [log for log in logs if 'eval_loss' in log and 'eval_accuracy' in log]

if eval_logs:
    # 同时根据 eval_loss 找到最优模型
    best_log = min(eval_logs, key=lambda x: x['eval_loss'])
    best_step = best_log['step']
    best_loss = best_log['eval_loss']
    best_accuracy = best_log['eval_accuracy'] # 同时获取最佳点的准确率
    logger.info(f"找到最佳模型在第 {best_step} 步，其验证集损失为: {best_loss:.4f}，准确率为: {best_accuracy:.4f}")

    best_checkpoint_path = os.path.join(training_args.output_dir, f"checkpoint-{best_step}")

    if os.path.exists(best_checkpoint_path):
        logger.info(f"正在从最佳检查点 '{best_checkpoint_path}' 加载模型...")
        model, tokenizer = FastLanguageModel.from_pretrained(
            model_name=best_checkpoint_path,
            load_in_4bit=True,
        )
    else:
        logger.warning(f"未找到最佳检查点路径 '{best_checkpoint_path}'，将使用训练结束时的模型。")
else:
    logger.warning("未找到任何评估日志，将使用训练结束时的最终模型。")

# 使用已加载的最佳模型进行保存
FINAL_SAVE_PATH = "/hy-tmp/Qwen3-14b-finetuned"
logger.info(f"正在保存最佳模型到 '{FINAL_SAVE_PATH}'")
model.save_pretrained(FINAL_SAVE_PATH)
tokenizer.save_pretrained(FINAL_SAVE_PATH)
logger.info(f"最佳 LoRA 适配器和分词器已保存。")

MERGED_SAVE_PATH = "/hy-tmp/Qwen3-14b-finetuned-merged"
logger.info(f"正在合并最佳 LoRA 权重并保存完整模型到 '{MERGED_SAVE_PATH}'...")
model.save_pretrained_merged(MERGED_SAVE_PATH, tokenizer, save_method="merged_16bit")
logger.info(f"已合并权重的完整模型已保存。")

2025-06-15 02:11:03 - INFO - [2573958225.py-16] - [<module>] - 开始模型微调流程
2025-06-15 02:11:03 - INFO - [2573958225.py-24] - [<module>] - 已加载数据集，包含 5000 个样本
2025-06-15 02:11:03 - INFO - [2573958225.py-31] - [<module>] - 正在从 /hy-tmp/Qwen3-14b 加载模型，最大序列长度: 512


==((====))==  Unsloth 2025.6.2: Fast Qwen3 patching. Transformers: 4.52.4.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 47.499 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

2025-06-15 02:12:33 - INFO - [2573958225.py-38] - [<module>] - 模型和分词器加载成功
2025-06-15 02:12:33 - INFO - [2573958225.py-41] - [<module>] - 正在配置 LoRA 适配器
2025-06-15 02:12:41 - INFO - [2573958225.py-54] - [<module>] - LoRA 适配器配置完成


Map:   0%|          | 0/4500 [00:00<?, ? examples/s]

...

Unsloth: Merging weights into 16bit:  75%|███████▌  | 6/8 [00:57<00:20, 10.12s/it]

Copied model-00002-of-00008.safetensors from local model directory


Unsloth: Merging weights into 16bit:  88%|████████▊ | 7/8 [01:07<00:10, 10.37s/it]

Copied model-00006-of-00008.safetensors from local model directory


Unsloth: Merging weights into 16bit: 100%|██████████| 8/8 [01:18<00:00,  9.84s/it]
2025-06-15 03:14:39 - INFO - [2573958225.py-170] - [<module>] - 已合并权重的完整模型已保存。
